In [1]:
print("Ok")

Ok


In [2]:
%pwd

'c:\\Users\\prasa\\Downloads\\End-to-end-Medical-Chatbot-Generative-AI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\prasa\\Downloads\\End-to-end-Medical-Chatbot-Generative-AI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File

def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [10]:
# Split the data into text chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [11]:
text_chunks=text_split(extracted_data)
print("Length of text chunks", len(text_chunks))

Length of text chunks 5773


In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

In [24]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [27]:

embeddings = download_hugging_face_embeddings()

c:\Users\prasa\anaconda3\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prasa\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling bac

In [28]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [30]:
#query_result

In [76]:
import os 
os.environ["PINECONE_API_KEY"] = "pcsk_2LhY5f_TRfueAJgY75j27pWMaBFW5oK6R4yvQCJjseEtT2YgwknPwa7KP3mVY62QySzZSX"
os.environ["COHERE_API_KEY"] = "36laOXHGcG9rxjNqgqyWTWvLVOPXs4GU3PAA9J7n"

In [50]:
#Embed each chunk and upsert the embeddings into Pinecone index.

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name="test",
    embedding=embeddings,
)

In [51]:
#Load Existing index

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name="test",
    embedding=embeddings 
)


In [52]:
docsearch

In [62]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [63]:
retrieved_docs = retriever.invoke("What is Acne?")

In [64]:
retrieved_docs


[Document(id='e2e59d48-1ed0-465d-9868-c22c31e1252b', metadata={'creationdate': '2004-12-28T15:13:17-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-28T14:23:27-06:00', 'page': 21.0, 'page_label': '22', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Alternative Medicine, 2nd Edition. Vol. 1, A-C.pdf', 'total_pages': 602.0}, page_content='GALE ENCYCLOPEDIA OF ALTERNATIVE MEDICINE 2 5\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nAcne\nDefinition\nAcne is a common inflammatory skin disease char-\nacterized by pimples on the face, chest, and back. It oc-\ncurs when the pores of the skin become clogged with oil,\ndead skin cells, and/or bacteria.'),
 Document(id='02999782-4643-4188-b974-6f4407440efd', metadata={'creationdate': '2004-12-28T15:13:17-05:00', 'creator': 'PyPDF

In [78]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [79]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [80]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [85]:
from langchain_cohere import ChatCohere
from langchain.chains import RetrievalQA

# Set your Cohere API key
import os
os.environ["COHERE_API_KEY"] = "36laOXHGcG9rxjNqgqyWTWvLVOPXs4GU3PAA9J7n"

# Initialize Cohere LLM
llm = ChatCohere(model="command-r")   # "command-r" or "command-light"

# Build RAG chain with your retriever/vectorstore
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=docsearch.as_retriever(),
    chain_type="stuff"
)

# Ask a question
response = rag_chain.invoke({"query": "What is Acne?"})
print(response["result"])


Acne is a skin disorder that causes inflammation of the sebaceous glands. It commonly appears as pimples on the face, chest and back, but can also affect other areas of the body. It occurs when the pores of the skin become blocked by oil, dead skin cells and/or bacteria, leading to irritation and inflammation. This condition is often referred to as acne vulgaris.


In [86]:
response = rag_chain.invoke({"query": "What is the treatment for acne?"})
print(response["result"])

Mild acne can be treated with over-the-counter medications containing ingredients like benzoyl peroxide, salicylic acid, sulfur, or resorcinol. These can help reduce the formation of new comedones. More severe cases of acne require prescription medications supervised by a doctor, such as comedolytics (which unclog pores), antibiotics (for up to three months), or isotretinoin, a powerful drug reserved for severe cases. Isotretinoin can be very effective, but it requires caution for women who may become pregnant.


In [87]:
response = rag_chain.invoke({"query": "What is pimple?"})
print(response["result"])

A pimple is a small inflamed bump on the skin, it occurs when the pores of the skin become clogged with oil, dead skin cells, and/or bacteria. Pimples are often associated with acne and usually appear on the face, chest, and back.


In [88]:
response = rag_chain.invoke({"query": "What is inflamation?"})
print(response["result"])

Inflammation is the body's response to infection or injury. It is a natural process that occurs when the body's immune system tries to protect itself from harm. During inflammation, the body's immune cells and proteins are activated and rush to the site of injury or infection. This results in the classic signs of inflammation: redness, heat, swelling, and loss of function. 

The purpose of inflammation is to eliminate the causative agent, clear out dead and damaged tissue, and initiate tissue repair. It's a crucial part of the body's healing process. However, it can also be a complex process that can sometimes cause damage if it's not properly controlled. Different types of inflammation exist, ranging from acute inflammation that lasts for a few days to chronic inflammation that can persist for months or even years.
